# Find Followees on Twitter

## 1. Introduction and Problem Statement

Social networking is becoming an indispensable part for us, which has made our lives easier, more efficient and convenient. Users can post and share their ideas with each other on social media, at the same time, get news and updates from different fields. Twitter, a popular and powerful social networking tool which is used by billions people every single day, provides free service to users and connects people from all over the world. As a social media application, it is important to help users find the information they need and follow the people or things they are interested in.

Although users can find their "followees" from hot topics or their friends, we sometimes cannot find the one we really want to follow because Twitter don't display more enough information about the guys we haven't followed. Now, we'd like to design a twitter recommender system to help users to find the followee they are interested in and may follow in the future.

In this project, we implemented content-based and collaborative filtering approaches separately to do user recommendation.  
  
For the hybrid content-based system, firstly we construct a user mentioning graph by what the user mentions in its tweets. Based on the graph, we calculate a score for target user and common users as a baseline of next step. Then we extract the tweets and analyze their similarities. Finally, we combine them together to get a score list to estimate which user should be recommended.  

For the collaborative filtering system, especially in this topic, we cannot get an ideal result from the old CF method with jaccard/cosine to calculate the similarity because the relationship between users is only follow/unfollow (no ratings),  we build a user following graph and do PageRank[1] on the graph to rank the popularity of the nodes. According to the popularity rank, we calculate the similarity between the users and recommend the target user with the followees of his/her similar users.  



## 2. Related Work:

Collaborative filtering recommendation system is based on the idea that similar users will do similar performance. Early work on collaborative filtering only put same weights on every users and calculate the similarity using Jaccard or Cosine[2]. Later work separate the system into several parts and use hybrid strategy to find the similar users[3]. However, the strategy still regards each user as same node and do not care more about the latent difference among individuals. In this project, we mainly use pageRank to get the popularity difference between users and calculate the similarity based on this.

Content-based recommendation systems try to recommend items similar to those a given user has liked in the past.[4] For the early work, the basic process performed by a content-based recommender consists in matching up the attributes of a user profile in which preferences and interests are stored, with the attributes of a content object, in order to recommend to the user new interesting items. Purely content-based is no need for data from other users which means there will be no cold-start problem for it. However, it’s hard for it to find out appropriate features. And due to it, purely content-based recommender always has a lower accuracy than collaborative filtering. So based on the traditional content-based algorithm, we add some new features based on observation and experiments. And by combining with user mentions, we try to promote the recommender even with a limited dataset. 


## 3. Data Collection:
When this project was first conceived, we planned on using twitter as our main source of data. However, don’t know when, twitter began to limit the open resource of tweets. It’s impossible to get completely existing tweets data. And the API of twitter is extremely unfriendly for users with so many limitations. 

So we can only get user relationship graph from SNAP (for the first part) and the tweets data from homework 2 (for the second part). Even so, the user following data is so huge (around 25GB) to deal with, so we have to extract a small part of it for our algorithm to use. Since the data is from around 4 years ago, so some of users may have deactivated or closed their accounts. That also leads us some troubles. So in the evaluation of part 2, we will not use the those kind of IDs to test our algorithm.


## 4. Methods

### 4.1 Hybrid Content-based Recommend System
For the hybrid content-based system, firstly we construct a user mentioning graph by what the user mentions in its tweets. Based on the graph, we calculate a score for target user and common users as a baseline of next step. Then we extract the tweets and analyze their similarities. Finally, we combine them together to get a score list to estimate which user should be recommended.  

### 1) Construct Mention Graph and Extract Tweets

For each user mentioning, we construct a dictionary to store their mentioning relationship. And for the next step, we also extract the user tweets at this part. 

In [2]:
from __future__ import division
import numpy as np
import os
import re
import json
import copy

graph = {}
mentionedGraph = {}
defaultRec = {}
baseScore = []
privateScore=[]
nodeIndex = {}
stopList = []
id2Name = {}
name2ID = {}
contentByID = {}
#scoreByID = {}

tempContent = []
f = open('english.stop')
for line in f:
    tempContent.append(line)
f.close()
stopList=set(('\n'.join(tempContent)).split())

def constructGraph():
    inputFile = open('pagerank.json', 'r')
    tweets = inputFile.readlines()
    for content in tweets:
        data = json.loads(content)
        userId = data['user']['id']
        name = data['user']['screen_name']
        if userId not in id2Name:
            id2Name[userId]=name
        if name not in name2ID:
            name2ID[name]=userId
        if userId not in graph.keys():
            graph[userId] = []
        for mention in data['entities']['user_mentions']:
            mentionedName = mention['screen_name']
            mentionedID = mention['id']
            if mentionedID not in id2Name:
                id2Name[mentionedID]=mentionedName
            if mentionedName not in name2ID:
                name2ID[mentionedName]=mentionedID
            if mention['id'] != userId:
                if mention['id'] not in graph:
                    graph[mention['id']] = []
                if mention['id'] not in graph[userId]:
                    graph[userId].append(mention['id'])
                if mention['id'] in mentionedGraph:
                    if userId not in mentionedGraph[mention['id']]:
                        mentionedGraph[mention['id']].append(userId)
                else:
                    mentionedGraph[mention['id']] = [userId]
    print 'User Graph loaded'
    inputFile.close()
#constructGraph()
#print graph
#for node in graph:
#    if len(graph[node])>7:
#        print node, graph[node], '\n'


### 2) Pagerank and Baseline

Based on user mentioning graph, we run the pagerank algorithm to get the baseline score of common recommendation, so we can recommmend for new users. Furthermore, we allow to introduce a traget ID to calculate the private score for that user. 

In [4]:
def PageRanker(targetID):
# clean empty nodes
    tempSet = set()
    for node in graph.keys():
        if not graph[node]:
            tempSet.add(node)
    for node in tempSet:
        if node not in mentionedGraph.keys() or not mentionedGraph[node]:
            del graph[node]
            
    # build index of node
    num = 0
    for node in graph:
        nodeIndex[node] = num
        num += 1
        
    # build Transfer Matrix  
    length = len(graph)
    d = 0.9
    linkMat = np.zeros((length, length))
    teleMat = np.full((length,length), (1/length))
    row = 0
    for node in graph.keys():
        mentionedList = []
        for mentionedNode in graph[node]:
            mentionedList.append(nodeIndex[mentionedNode])
        if len(mentionedList) == 0:
            j = 1/length
            for x in range(length):
                linkMat[row,x] = j
        else:
            j = 1/len(mentionedList)
            for x in mentionedList:
                linkMat[row,x] = j
        row += 1

    tranMat = d * linkMat + (1-d) * teleMat

    #make the default recommendations for new users
    # calculate pr in iteration times
    #iteration = 200
    pr = np.full((1,length),(1/length))
    #pr = np.zeros((1,length))
    #pr[0][nodeIndex[1514379421]] = 1
    c = 1
    while c >= 0.1:
        pr1 = pr
        pr = np.dot(pr,tranMat)
        c = abs(np.dot(pr1, pr.T) - np.dot(pr,pr.T))
    

    # Rank result
    global baseScore
    result = pr.tolist()[0]
    baseScore = copy.deepcopy(result)
    scoreList = [0.0 for i in range(5)]
    rankList = [0 for i in range(5)]
    k = 0
    for node in graph.keys():
        score = result[k]
        for l in range(5):
            if score > scoreList[l]:
                rankList[l] = node
                scoreList[l] = score
                break
        k += 1
    for m in range(5):
        defaultRec[rankList[m]] = scoreList[m]
        
    #begin to calculate the private score
    if targetID in nodeIndex:
        pr = np.zeros((1,length)) 
        pr[0][nodeIndex[targetID]] = 1
        c = 1
        while c >= 0.1:
            pr1 = pr
            pr = np.dot(pr,tranMat)
            c = abs(np.dot(pr1, pr.T) - np.dot(pr,pr.T))
    
        global privateScore
        result = pr.tolist()[0]
        privateScore = copy.deepcopy(result)
    
    """
    # Rank result
    scoreList1 = [0.0 for i in range(10)]
    rankList1 = [0 for i in range(10)]
    k = 0
    for node in graph.keys():
        score = result[k]
        for l in range(10):
            if score > scoreList1[l]:
                rankList1[l] = node
                scoreList1[l] = score
                break
        k += 1
    
  
    # Test Result
    print 'user ID - Score'
    for m in range(5):
        print '%s - %f' %(rankList[m], scoreList[m])
    print defaultRec

    print 'user ID - Score'
    for m in range(10):
        print '%s - %f' %(rankList1[m], scoreList1[m])
        
    """  

### 3) Content Preprocess

In this part, we tokenize user tweets in word. Since we need to figure out the similarity of user contents, so we eliminate stop words and URL in text by regular expression. 

In [10]:
def filterStopWords(words):
    """Filters stop words."""
    filtered = []
    for word in words:
        if not word in stopList and word.strip() != '':
            filtered.append(word)
    return filtered


def contentBased():
    inputFile = open('pagerank.json', 'r')
    tweets = inputFile.readlines()
    for content in tweets:
        data = json.loads(content)
        userId = data['user']['id']
        words = data['text'].lower()
        #text='RT @KissMeTheVamps: @TheVampsBrad  @TheVampsband @TheVampsJames @TheVampsCon @TheVampsBrad @TheVampsTristan http:\\/\\/t.co\\/14cow6BScW MY FAVOR\u2026'
        words = re.sub('(http:\\\\[\/\\\.a-z]+)', '', words)
        words = filterStopWords(words.split())
        if userId not in contentByID:
            contentByID[userId] = {}
        for word in words:
            if word in contentByID[userId]:
                contentByID[userId][word] += 1
            else:
                contentByID[userId][word] = 1
    
    
#use Manhattan distance to calculate 


#print len(contentByID)
#print len(mentionedGraph[493249666]), len(mentionedGraph[618336955]), len(mentionedGraph[30974408]), len(mentionedGraph[1479930294]),len(mentionedGraph[1481176242]),len(mentionedGraph[16510838])
#print len(mentionedGraph[1056749292]), len(mentionedGraph[495074577]), len(mentionedGraph[126370504]), len(mentionedGraph[629550615]),len(mentionedGraph[1364831238]),len(mentionedGraph[197759552])

### 4) Recommendation Calculation

In this part, we tried many method to calculate the similarity such as jaccard and manhattan distance. However, we expect the result can take the times of word appearance into consideration and it should be easy to normalized in a certain range. So we use cosine to determine the similarity of two tweets. After that, we sum up the corresponding user baseline score with their similarity score and store it as part of user profile. Then we adapt the parameters based on observation and experiment result. For the number of followers in certain range, we add additional weight for that user. If users follow each other, they will have higher weight for recommendation. And then we sort the result and output them. 

In [9]:
def cosine(a,b):
    tempScore = 0.0
    a2 = 0.0
    b2 = 0.0
    c=list(set(a.keys()+b.keys()))
    for word in c:
        if word not in a or word not in b:
            continue
        else:
            tempScore += a[word]*b[word]
    for a1 in a:
        a2 += a[a1]*a[a1]
    for b1 in b:
        b2 += b[b1]*b[b1]
    tempScore = tempScore / a2 / b2
    return tempScore


def recommend(targetID):
    PageRanker(targetID)
    if targetID in graph:
        scoreByID = {}
        for user in nodeIndex:
            if user != targetID:
                score = baseScore[nodeIndex[user]] + privateScore[nodeIndex[user]]
                #use cosine to calculate the score of content-based part 
                if user in contentByID:
                    score += cosine(contentByID[targetID], contentByID[user])
                if user in mentionedGraph:
                    if len(mentionedGraph[user]) >= 15:
                        score += 0.0015
                    if len(mentionedGraph[user]) <= 110:
                        score += 0.0005
                    if len(mentionedGraph[user]) >= 250:
                        score += 0.0005
                    if len(mentionedGraph[user]) >= 20 and len(mentionedGraph[user]) <= 60:
                        score += 0.0005
                    if targetID in graph and user in graph[targetID]:
                        score += 0.0015
                    if user in graph and targetID in graph[user]:
                        score += 0.0005
                scoreByID[user] = score
        scoreList = sorted(scoreByID.items(), key=lambda d:d[1], reverse = True)
        print 'Recommendations for user %s,  user ID:%d' %(id2Name[targetID], targetID)
    else:
        scoreList = sorted(defaultRec.items(), key=lambda d:d[1], reverse = True)
        print 'Recommendations for user %d' %targetID
    j=0
    print 'user ID - User Name'
    for node in scoreList:
        if j<5:
            print '%s - %s' %(node[0], id2Name[node[0]]) 
            j+=1
        else:
            break


constructGraph()
contentBased()

User Graph loaded


In [ ]:
inputID = raw_input('Please input the user ID you want to recommend for:')
recommend(int(inputID))

### 5) Approach and Methods
For the second part, we ever thought about several ways to combine the result of two parts. Since in the very beginning, we use manhattan to calculate the similarity and it's actually too hard to contral the range of its score. The first idea is use a coefficient to limit it. However, tweets of users are too different with each other. The length of them drove me crazy. Then we noticed the recommendation part for movies of our class. So we decided to try the baseline thought and it works well. But the normalization problem was still not solved. Actually, when I found that the cosine method had no need to normalize, I thought I was an idiot (the easiest way works the best T_T). Another interesting discovery is that we found that the popular user sometimes didn't followed by target user in our dataset. On the contrary, some common users, whose number of followers was around 50, had higer probability to be followed by another common user. That promotes our precision a lot. 

As we discussed at the beginning, the biggest problem is the data collection. It's a hard time to search for it and finally found that the twitter had already limited open resource of tweets for several years. And we tried to write script to capture tweets by ourselves. However, twitter is a bad company which didn't like to support research. Its limitation of API is so harsh that we were blocked with only 15 commands. But it's actually not a bad experience, since we learned a lot of how to capture data by ourselves, even which was not very successful.

### 6) Recommendation Result and Evaluation

Since our data is limited, so I randomly pick up 10 users whose tweets are more than 5. For each target user, I recommend 5 users for him or her. Because our data is from four years ago, I check whether our recommendations are valuable by checking the current followings of corresponding user.   

I list the recommendation results for users as follows and mark & for good recommendations which are really followed by target user now. Then we evaluate the recommend algorithm by this result.  

To determine whether one user is following another one, we use Twitter API to determine it. We need to submit request on the console:
https://apigee.com/console/twitter

The command is as follow:

https://api.twitter.com/1.1/friendships/show.json?source_id=[ID_1]&target_id=[ID_2]  

Replace the ids of two users in the [ID_1] and [ID_2] as follow and the API will return the result of their relationship.  

https://api.twitter.com/1.1/friendships/show.json?source_id=1514379421&target_id=493249666
______________________________________________________________________________________________________________________________________________  
  
Recommendations for user NoticedTheVamps,  user ID:1514379421  
user ID - User Name  
197759552 - spacehero1995
495074577 - TheVampsCon
493249666 - TheVampsTristan &
30974408 - TheVampsJames &
126370504 - TheVampsBrad &  


Recommendations for user lawrencedale_,  user ID:529244499  
user ID - User Name  
158314798 - Real_Liam_Payne &      
72064417 - PointlessBlogTv    
209708391 - onedirection  &     
105119490 - NiallOfficial &    
982345171 - ploynthanida  

Recommendations for user _Anklebiters_,  user ID:189690616  
user ID - User Name  
21111883 - ddlovato &    
43003845 - paramore &  
408774041 - DiannaDeLaGarza  
461410856 - TheParamoreBand & 
40981798 - yelyahwilliams &

Recommendations for user louismidnight,  user ID:634024610  
user ID - User Name  
440205732 - pukestagram  
431834660 - hushstyIes  
64259689 - harrynstuff &  
1425312570 - niallerlust &  
525737876 - weyheyygabi &   

Recommendations for user unfziam,  user ID:285410994
user ID - User Name
523753678 - nachoniall
158314798 - Real_Liam_Payne &  
72064417 - PointlessBlogTv
209708391 - onedirection &  
105119490 - NiallOfficial &   

Recommendations for user FifthharmonyWOW,  user ID:487491990  
user ID - User Name  
158314798 - Real_Liam_Payne  
72064417 - PointlessBlogTv  
209708391 - onedirection &  
105119490 - NiallOfficial  
14268057 - wittynate   

Recommendations for user McLeeroyFlurry,  user ID:1334763498  
user ID - User Name  
956559020 - Updating1DInfo  
612924569 - 1DAlert &  
267333181 - 1DSuperHumans  
477919735 - 1dupdategirls &  
591743479 - WW1DUpdates &  

Recommendations for user SMASHWORLD_OFC,  user ID:609804455  
user ID - User Name  
211089590 - SMASHindonesia &  
272400533 - ZulviaNazma  
326714055 - SMASHFUNIA  
257840821 - SMASH_TPI  
215245796 - SMASHBANDUNG  

Recommendations for user lifeloverforeva,  user ID:1417445364  
user ID - User Name
158314798 - Real_Liam_Payne &
269498821 - tickledpayne
105119490 - NiallOfficial &
534774305 - ShipNmily &
346883535 - OliveiraLais13  

Recommendations for user jazzy_jass123,  user ID:827443188  
user ID - User Name   
308297673 - KianLawley &  
45494617 - TrevorMoran  
308759071 - SamPottorff &  
139539038 - jccaylen &  
180949358 - ConnorFranta  


**Evaluation**  
Total Recommendations: 50   
Valid Recommendations: 27  
Precision: 54%  


### 4.2 Collaborative filtering
![](https://lh3.googleusercontent.com/-vcyfyBBpWnjO9Cz2Fb3DetcqAmyGjTpaLFsnGSYuYz9WT0yv3bY8kQFqpm2-A2YnTfGw6DLpAZj4YX9yrRlmaDbazTayzIJ3DaXk3ULGVPb6ccwONPsLMKH0OSl18U_tABA6LMfMHZdztqIXhwjMmd1zq4ZUOoeUH23vzhE6JMf_rgjqiKoWFz-DSuI5cyXu90xPxwX0WiK4UD3egKlDe06SNSAxW68fvGhr6Jk2J03QuY5yRq51uvzNp288eiPgfICiaY0EJ0sOxm1skEcDWIWAIGCQj05RKVewPmBcI4c-_ZIiS3ldHZi15-poei3nwxWvPE9VlqKKDDnT1XvrsN0ZqroKl4IWc_JeJ2Ow8kj-bhmLX9dM3cDUmmCYlQEaXpAHGk5Rtv1qvqOcqEZG8ecrrz1eH2gDmlJA8ZrdFlqFrdVB-qfVXVY7r5BMKIcgVd_ZlLeq6oPBaEjnblDkN00Dshh1wr_nL654vol4__YzaDTyDXoKoYpWWg3ffKfj2fHO4pMUq7ddFgRXC6AIlcM7Iu6WJCL6fNhm8SXj0E_kgxNcu2P-aU2Dqmaz17FQSuhbX1f6RCvbmGxY199fXs4Uxt_PR1KYxyEwpaUDnp8DrTkyV6G=w945-h198-no)

### 1) build the graph
According to the dataset of following relation http://snap.stanford.edu/data/higgs-social_network.edgelist.gz  
We build the followGraph and followedGraph to record all relationship among users

**Example**:  
A follow B and C  
B follow C and D  
C follow D and E  
  
In followGraph, data is presented like this:  
{ A->[B,C], B->[C,D], C->[D,E] }  
In followedGraph, data is presented like this:  
{ B->[A], C->[A,B], D->[B,C], E->[C] }

In [1]:
followMap = {}
followedMap = {}
userSet = set()
with open("higgs-social_network.edgelist") as f:
    lines = f.readlines()
    for line in lines:
        follow = line.split()
        #build followMap
        if follow[0] in followMap:
            followMap[follow[0]].append(follow[1])
        else:
            followMap[follow[0]] = [follow[1]]
        #build followedMap
        if follow[1] in followedMap:
            followedMap[follow[1]].append(follow[0])
        else:
            followedMap[follow[1]] = [follow[0]]
        #build userSet
        if follow[0] not in userSet:
            userSet.add(follow[0])
        if follow[1] not in userSet:
            userSet.add(follow[1])

### 2) build training data
To evaluate our recommender, we need to separate the dataset into two parts, training data and testing data. What we do is separate the first 1000 users' following list and extract the half the following list as the testing data of these users. We will compare the predict followees with these testing data to evaluate our system's accuracy.

For example:  
A is one of first 1000 users and A follow [B, C, D, E]  
then the following list will be separated into two parts, [B, C] (training data) and [D, E] (testing data)

In [2]:
# select the first 1000 user as test user
# remove half of their followees and use them as predict objects
import copy
trainFollowMap = copy.deepcopy(followMap)
trainFollowedMap = copy.deepcopy(followedMap)

n = 0
for user in trainFollowMap:
    if n < 1000:
        followees = trainFollowMap[user]
        trainFollowMap[user] = followees[0:len(followees)/2]
        for followee in followees[len(followees)/2:]:
            trainFollowedMap[followee].remove(user)
            if not trainFollowedMap[followee]:
                del trainFollowedMap[followee]
    else:
        break
    n += 1

### 3) Implement PageRank on the graph
We implement PageRank on the graph to get the popularity rank of users (10 iteration). The popularity is defined as the possibility of being followed by others.

In [3]:
#pageRank to get popularity rank
popularity = {}
d = 0.8
Num = len(userSet)
iterNum = 10
for user in userSet:
    popularity[user] = 1.0 / len(trainFollowMap)
lastPop = popularity.copy()

for i in range(iterNum):
    sumOfPop = 0
    for user in userSet:
        if user in trainFollowedMap:
            followers = trainFollowedMap[user]
        else:
            followers = []
        popularity[user] = (1-d)/Num
        for follower in followers:
            popularity[user] += d * lastPop[follower] / len(trainFollowMap[follower])
        sumOfPop += popularity[follower]
    #normalize popularity
    for user in popularity:
        popularity[user] /= sumOfPop
    lastPop = popularity.copy()

### 4) Inverse popularity rank
The similariy is defined as the possibility of following the same followees. To find the similar users having similar followee list with the target user, we have a basic idea:  
**If two users follow the same followee with lower popularity, the more likely these two are similar**

In [4]:
#inverse popularity rank
ivs_popularity = {}
popOfSum = 0
for user in popularity:
    ivs_popularity[user] = 1.0/popularity[user]
    popOfSum += ivs_popularity[user]
for user in popularity:
    ivs_popularity[user] /= popOfSum

### 5) Similarity Function
As defined above, The similarity of two users is calculated by the cosine of follow vectors and the weight of user is based on the **inverse popularity rank**    

We also test the similarity function calculated by **popularity rank** and **jaccard **  

In [5]:
def simByIvsPopularity(userA, userB):
    common = list(set(trainFollowMap[userA]).intersection(trainFollowMap[userB]))
    if not common:
        return 0
    numerator = 0
    for user in common:
        numerator += ivs_popularity[user] ** 2
    mode = 0
    denominator = 1
    for user in trainFollowMap[userA]:
        mode += ivs_popularity[user] ** 2
    denominator *= mode ** 0.5
    
    mode = 0
    for user in trainFollowMap[userB]:
        mode += ivs_popularity[user] ** 2
    denominator *= mode ** 0.5
    return numerator / denominator

def simByPopularity(userA, userB):
    common = list(set(trainFollowMap[userA]).intersection(trainFollowMap[userB]))
    if not common:
        return 0
    numerator = 0
    for user in common:
        numerator += popularity[user] ** 2
    mode = 0
    denominator = 1
    for user in trainFollowMap[userA]:
        mode += popularity[user] ** 2
    denominator *= mode ** 0.5
    
    mode = 0
    for user in trainFollowMap[userB]:
        mode += popularity[user] ** 2
    denominator *= mode ** 0.5
    return numerator / denominator

def simJaccard(userA, userB):
    common = list(set(trainFollowMap[userA]).intersection(trainFollowMap[userB]))
    if not common:
        return 0
    return len(common) / (len(trainFollowMap[userA]) + len(trainFollowMap[userB]) - len(common))


### 6) Evaluation
To evaluate the system, we will compare the predict followees with these testing data to evaluate our system's accuracy.  
In detail, we selected the top 100 users having highest similarity with the target user and calculate their followees frequency and select top 5 of them as the result of the recommender. We totally test 100 users and get the precision of our recommender system and compare it with other system using other similarity function.

In [14]:
import heapq
import operator
limit = 100
count = 1
predictCorrect = 0
predictTotal = 0
for userA in trainFollowMap:
    print "userId: ",userA,str(count)+ "/" + str(limit)
    h = []
    numOfUser = 100
    for userB in trainFollowMap:
        if userA != userB:
            heapq.heappush(h, (simByIvsPopularity(userA, userB), userB))
            if len(h) > numOfUser:
                heapq.heappop(h)
    followeeList = {}
    for e in h:
        user = e[1]
        for followee in trainFollowMap[user]:
            if followee not in trainFollowMap[userA]:
                if followee in followeeList:
                    followeeList[followee] += 1
                else:
                    followeeList[followee] = 1
    sorted_list = sorted(followeeList.items(), key=operator.itemgetter(1),reverse=True)
#     predict = [i[0] for i in sorted_list[0:(len(followMap[userA])+3)/4]]
    if len(sorted_list) < 5:
        predict = [i[0] for i in sorted_list]
    else:
        predict = [i[0] for i in sorted_list[0:5]]
    real = followMap[userA][len(followMap[userA])/2:]
    common = list(set(predict).intersection(real))
    predictCorrect += len(common)
    predictTotal += len(predict)
#     print "predict: ",predict
#     print "real: ",real
#     print "common: ", common
#     print "accuracy:", len(common) / float(len(predict))
    if count == limit:
        break
    count += 1

userId:  378466 1/100
userId:  287145 2/100
userId:  287146 3/100
userId:  287147 4/100
userId:  378462 5/100
userId:  378463 6/100
userId:  287142 7/100
userId:  287143 8/100
userId:  287148 9/100
userId:  378464 10/100
userId:  378468 11/100
userId:  378469 12/100
userId:  370255 13/100
userId:  370254 14/100
userId:  370257 15/100
userId:  378465 16/100
userId:  370251 17/100
userId:  370250 18/100
userId:  370253 19/100
userId:  370252 20/100
userId:  89378 21/100
userId:  89379 22/100
userId:  370259 23/100
userId:  370258 24/100
userId:  287141 25/100
userId:  5988 26/100
userId:  5989 27/100
userId:  378467 28/100
userId:  378460 29/100
userId:  5982 30/100
userId:  5983 31/100
userId:  5980 32/100
userId:  5981 33/100
userId:  5986 34/100
userId:  5987 35/100
userId:  5984 36/100
userId:  5985 37/100
userId:  218135 38/100
userId:  218134 39/100
userId:  287149 40/100
userId:  391271 41/100
userId:  162929 42/100
userId:  89370 43/100
userId:  89371 44/100
userId:  89372 45/100

In [16]:
print predictCorrect
print predictTotal
print "precision: ", predictCorrect / float(predictTotal)

98
500
precision:  0.196


### 7) Result
We compare the precisions of recommenders with three similarity functions (jaccard, popularity, inverse-popularity) and get the following result:  

jaccard: 0.0039603960396  
popularity: 0.11  
inverse-popularity: 0.196  

<img src="https://lh3.googleusercontent.com/2mfidqBWqnGBZx5AopfC5MiRUCK7HTIeSN9B-y-HEl4PEmFAGwViD2Z-zz5_IWGDGY4PPQj_gX26Y7EWCKLxEiApiUiRC2x3pRHDJjJP3cZCyaLjWq9_IC0M0rmFu_jJ9avUVVWMU1YOirnjzTGEgWIqvfHYcY-vWVFfu_AtLi77BsoJN0_5_zKEg09-Lk6yPd0k2oPhgEgrJclgq5d8SUHRqoFx4BZkDR1PwmHy-3q-HvegDDS_TusQ0zF95Zf9kKtOLvJe5aRhQSRjTEwp33rFNCX9Qi9NdpSMgxGEPIibfMKMC2RfcwIGExkVYTaropvyssvgJJISxO9f06odJ9r-KQqil3IiEvRB7olE1XcWIqNIhI-MUvLSGCPC_JREqRG0dWXhlILzzwr5X86dP8md-FxrMU4-Sz8Cg7SEsKxcufMXEdDEtxlnHMHAE1vpdddBJs9FIvaxK6zqbFXqvxnRUv_DxpsBrgi_qvEMIg6sLUHzJSMMzWzOfnyVwP7izr6rw7XO-ak78f3LJEMu0eSF3_ppZ4Cc-ZkBpPZopUrL5lQ_VB602G0bwiBjfysDEoZEjQGeljQoimr0Ryn6rBL11M3ZI2uawpujTNhUeQhBJlg61xYu=w929-h535-no" width="500">


## 5. Conclusion:
In short, for recommender system, users’ hobbies and interests are indeed a hard part to predict. In the collaborative filtering part, we found that if two people concerned about similar topics, they actually would not follow each other. However, if their interests are similar, it makes sense that the rest followings of one user is interesting for the other one, which is just like what we learn for recommending movies. We use different methods to calculate similarities of users, such as jaccard and cosine, and find out our best way to solve this problem, the inverse popularity ranking. 

For the hybrid content-based part, we found that the baseline is very important for recommendation. If a user mentioned one people in a topic, what that guy concerned about would also mean a lot to the target use. And by observation and experiment, we also found that it’s not true that the more popular a user is, the more likely it would be followed.The user who is more likely followed is always a normal user which means the number of his or her followers is usually in a certain range around 50-150. And ignoring stop word and URL in user’s posts also helps to promote our accuracy. For this part, we use tokenization technique to preprocess tweets and pagerank algorithm to calculate the baseline of recommendation score. And adjusting our parameters based on the results of experiments.

Due to the limitation of data, our result is not as good as what is said in paper. But we just wanna try to realize and add some of our ideas in the existing algorithm and then figure out what we can do. 

For the next step, we think we will redefine each user with their profile and improve the similarity algorithm and mark every one with a topic by semantic analysis if with more tweets data for the content-based part.

## Reference
[1] Page L, Brin S, Motwani R, Winograd T. The PageRank citation ranking: Bringing order to the web. Stanford InfoLab; 1999 Nov 11.  
[2] Breese, John S., David Heckerman, and Carl Kadie. "Empirical analysis of predictive algorithms for collaborative filtering." Proceedings of the Fourteenth conference on Uncertainty in artificial intelligence. Morgan Kaufmann Publishers Inc., 1998.  
[3] Hannon, John, Mike Bennett, and Barry Smyth. "Recommending twitter users to follow using content and collaborative filtering approaches." Proceedings of the fourth ACM conference on Recommender systems. ACM, 2010.   
[4] Lops, Pasquale, Marco De Gemmis, and Giovanni Semeraro. "Content-based recommender systems: State of the art and trends." Recommender systems handbook. Springer US, 2011. 73-105.  
